In [81]:
import numpy as np


import string
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from pyspark.ml.clustering import KMeans, KMeansModel, KMeansSummary
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel, IDF, IDFModel
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, BooleanType, StructType, StructField
from pyspark.ml import Pipeline, PipelineModel

from nlp_cl_start import data_tokenizer

In [2]:
import pandas as pd

In [3]:
import pyspark as ps
spark = (ps.sql.SparkSession.builder
        .master("local[4]")
        .appName("yelp_academic")
        .getOrCreate()
        )
sc = spark.sparkContext

In [17]:
#x = PipelineModel.read()

In [24]:
spark

In [120]:
#p = x.load('tfidfkmeans/')
km1 = KMeansModel.load('kmeans/')

RuntimeError: No training summary available for this KMeansModel

In [10]:
#pipe = PipelineModel.load('tfidf/')
pipe = PipelineModel.load('pipe-idf-5-7-18/')

In [6]:
#us_ex = pd.read_csv('user_from_scr_clidf18.csv')

In [12]:
#np.unique(us_ex.user_cl, return_counts=True)

(array([ 0,  1,  2,  4,  5,  7,  8,  9, 10, 11, 12, 13, 16, 17]),
 array([71, 81, 30, 47, 38, 20, 17, 22,  8, 20,  4, 10,  9,  8]))

In [5]:
from pymongo import MongoClient
import time
from bs4 import BeautifulSoup

mc = MongoClient()
db  = mc['raw_restaurants']
rv_s = db['reviews_scrap']

In [6]:
user_test = list(rv_s.find({'user_id': '_3KTfz0hcPl_yq4jOyOPiA'}))

In [7]:
u_df =pd.DataFrame(user_test)


In [8]:
df = pd.DataFrame([{'a':1, 'b':2 },{'a':9, 'b':20 },{'a':17, 'b':26 },{'a':16, 'b':2 }])
df

,a,b
0,1,2
1,9,20
2,17,26
3,16,2


In [9]:
ddd = spark.createDataFrame(df)

In [14]:
u_df.columns

Index(['_id', 'alias', 'category', 'date', 'id', 'rating', 'text', 'user_id'], dtype='object')

In [12]:
u_df2 = u_df.drop(columns=['_id', 'alias', 'category', 'date'])

In [13]:
u_df2.head()

,id,rating,text,user_id
0,ErsYo_9v8--feFCy92tymQ,5.0,Fantastic Thai food with warm service. Authent...,_3KTfz0hcPl_yq4jOyOPiA
1,W1NnSItt-SwWHrzkRtZ44g,5.0,Fantastic Vietnamese sandwich: great flavors a...,_3KTfz0hcPl_yq4jOyOPiA
2,3Q5uzROHYl1i2B_t2sIl7w,3.0,A fine dining restaurant that is in the midst ...,_3KTfz0hcPl_yq4jOyOPiA
3,-GYe-Aq7NcNJ3u-U6fmwMg,5.0,Great restaurant with an interesting mix of in...,_3KTfz0hcPl_yq4jOyOPiA
4,pPlsW81ntC2qbF0QOe6pLQ,5.0,We always have a good experience here with Ani...,_3KTfz0hcPl_yq4jOyOPiA


In [14]:
u_sp = spark.createDataFrame(u_df2)

In [15]:
from nlp_cl_start import data_tokenizer

In [17]:
u_token = data_tokenizer(u_sp)

In [18]:
u_pred = pipe.transform(u_token)

In [21]:
u_pred.columns

['id',
 'rating',
 'text',
 'user_id',
 'token',
 'vectors',
 'features',
 'prediction']

In [40]:
uu = u_pred.filter((u_pred.rating == '2.0')|(u_pred.rating == '1.0')).select( 'prediction').toPandas()

In [42]:
pd.unique(uu.prediction)

array([ 0,  1, 10])

In [43]:
from pipe_spar import  cluster_user_by_review

DataFrame[id: string, rating: string, text: string, user_id: string, token: array<string>]

In [44]:
up = cluster_user_by_review(u_token, pipe)

In [46]:
%%time
up.count()

CPU times: user 19.6 ms, sys: 8.93 ms, total: 28.6 ms
Wall time: 2.12 s


6

In [52]:
up.take(6)

[Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=4),
 Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=10),
 Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=7),
 Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=5),
 Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=0),
 Row(user_id='_3KTfz0hcPl_yq4jOyOPiA', user_cl=1)]

In [56]:
from scraping.add_by_scrap import scrap_by_users

In [58]:
kk= scrap_by_users('https://www.yelp.com/user_details?userid=KbU6Q0oy5X2mTShnRXiggg')

In [61]:
revs = spark.createDataFrame(kk)

/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [65]:
def new_user_clust_from_yelp(reviews):
    df = spark.createDataFrame(reviews)
    df = df.filter((df.rating == '2.0')|(df.rating == '1.0'))
    df = data_tokenizer(df)
    clusters = cluster_user_by_review(df, pipe)
    return clusters
    

In [66]:
cl = new_user_clust_from_yelp(kk)

/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [118]:
cl.first()

Row(user_id='KbU6Q0oy5X2mTShnRXiggg', user_cl=4)

In [68]:
r1 = """This review is based only on the drinks and service. 

Our group got seated at a booth. The decor and ambiance was pretty cool and trendy. When the server came to take our drink order, I had asked her what the Whiteclaw drink was listed under the Beer, Wine, etc. section of the menu. She had told me that it was like a cider that was a little sweet with a hint of grapefruit. That piqued my interest so I ordered that. 

When she brought our drinks, she placed a really tall can in front of me. I was pretty surprised and wasn't expecting a can, but it was fine. When I took my first sips, I barely tasted anything. I turned the can around to see the front side and noticed at the bottom it said, "Sparkling water with a hint of grapefruit". 

I was pretty confused and bummed out. I wasn't really sure why she would tell me that the Whiteclaw drink was cider. Cider and sparkling water are two very different drinks. 

I kept sipping and just tried to enjoy it since it was a huge $10 can. But after a few minutes, I decided to order some wine so I can actually enjoy myself. When a different server came to check up on us, my boyfriend asked if I could get the Anderson Valley Rosé. The server told us that that isn't actually a rosé but a completely different kind of drink. It was confusing, but they do actually have a rosé on the menu, which we ordered, but it's not the Anderson Valley drink listed on the menu. 

When our original server stopped by, she noticed I had the rosé and wasn't drinking my Whiteclaw sparkling water. She asked, "Did you not like it?". I said, "no, not really." And all she said was, "Well I like it." and walked away. Had she told me that it was sparkling water, I definitely wouldn't have ordered that and waste $10. 

Maybe this was just one of those off-chance, bad experiences. Maybe the food is good? The space is definitely pretty cool to hang with friends after dinner or for happy hour. Not sure if I'd return. There's a lot of other bars that have less confusing drink menus and better service that I would prefer going to.
"""

In [85]:
from nlp_cl_start import tokenize

In [87]:
token = tokenize(r1)

In [107]:
rw_df = spark.createDataFrame([{'token': token, 'user_id' :0},{'token': token, 'user_id' :0},{'token': token, 'user_id' :0} ])

/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/sql/session.py:340: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


In [108]:
g = cluster_user_by_review(rw_df, pipe)

In [109]:
ff = g.collect()

In [110]:
len(ff)

1

In [102]:
ff[0]

Row(user_id=0, user_cl=17)

In [114]:
def transform_users(cls):
    a = []
    for cl in cls:
        a.append('cl_t'+str(cl[1]))
    return a

In [119]:

transform_users(cl.collect())

['cl_t4', 'cl_t2', 'cl_t17']